In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.point import Point
import numpy as np

In [2]:
movies = pd.read_csv('rawdata/title.basics.tsv/data.tsv', sep='\t')
display(movies)
movies = movies[(movies.isAdult == '0') | (movies.isAdult == 0)]
movies = movies[movies.titleType == 'movie']
movies = movies.drop(columns=['titleType', 'originalTitle', 'isAdult', 'endYear'])
movies = movies.rename(columns={"tconst": "movieID", "primaryTitle": "title", "startYear": "year", "runtimeMinutes": "runtime"})
movies['title'] = movies.title.str.replace(r"\\N", '')
movies['year'] = movies.year.str.replace(r"\\N", '')
movies['runtime'] = movies.runtime.str.replace(r"\\N", '')

C:\Users\andyc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7703469,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7703470,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7703471,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7703472,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [3]:
genres = movies.set_index('movieID').genres.str.split(',', expand=True).stack().reset_index('movieID')
genres = genres.rename(columns={0: "Genre"})
genres = genres[genres.Genre != "\\N"]
display(genres)

,movieID,Genre
0,tt0000574,Action
1,tt0000574,Adventure
2,tt0000574,Biography
0,tt0000615,Drama
0,tt0000630,Drama
...,...,...
0,tt9916620,Drama
0,tt9916622,Documentary
0,tt9916680,Documentary
0,tt9916706,Comedy


In [4]:
movies = movies.drop(columns=['genres'])
budrev = pd.read_csv('rawdata/tmdb_movies_data.csv', sep=',')
keywords = budrev.drop(columns=['id','popularity','budget','revenue','original_title','cast','homepage','director','tagline','overview','runtime','genres','production_companies','release_date','vote_count','vote_average','release_year','budget_adj','revenue_adj'])
budrev = budrev.drop(columns=['id','popularity','original_title','cast','homepage','director','tagline','overview','runtime','genres','production_companies','release_date','vote_count','vote_average','release_year','keywords'])
display(keywords)
budrev = budrev.rename(columns={"imdb_id": "movieID"})
budrev = pd.merge(movies, budrev, how='left').replace({np.nan: ''})
budrev.to_csv('finaldata/movie.txt', sep='\t', index=False)
display(budrev)

,imdb_id,keywords
0,tt0369610,monster|dna|tyrannosaurus rex|velociraptor|island
1,tt1392190,future|chase|post-apocalyptic|dystopia|australia
2,tt2908446,based on novel|revolution|dystopia|sequel|dyst...
3,tt2488496,android|spaceship|jedi|space opera|3d
4,tt2820852,car race|speed|revenge|suspense|car
...,...,...
10861,tt0060371,surfer|surfboard|surfing
10862,tt0060472,car race|racing|formula 1
10863,tt0060161,car|trolley|stealing car
10864,tt0061177,spoof


,movieID,title,year,runtime,budget,revenue,budget_adj,revenue_adj
0,tt0000502,Bohemios,,100,,,,
1,tt0000574,The Story of the Kelly Gang,,70,,,,
2,tt0000615,Robbery Under Arms,,,,,,
3,tt0000630,Hamlet,,,,,,
4,tt0000675,Don Quijote,,,,,,
...,...,...,...,...,...,...,...,...
562006,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,57,,,,
562007,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,100,,,,
562008,tt9916706,Dankyavar Danka,2013,,,,,
562009,tt9916730,6 Gunn,2017,116,,,,


In [5]:
keywords = keywords.rename(columns={"imdb_id": "movieID"})
keywords = pd.merge(movies, keywords).replace({np.nan: ''})
keywords = keywords.drop(columns=['title', 'year', 'runtime'])
keywords = keywords.set_index('movieID').keywords.str.split('|', expand=True).stack().reset_index('movieID')
keywords = keywords.rename(columns={0: "keyword"})
display(keywords)
keywords.to_csv('finaldata/keywords.txt', sep='\t', index=False)

,movieID,keyword
0,tt0035423,lover (female)
1,tt0035423,love of one's life
2,tt0035423,time travel
3,tt0035423,kiss
4,tt0035423,past
...,...,...
4,tt5083702,legal system
0,tt6019206,sword
1,tt6019206,martial law
2,tt6019206,revenge


In [6]:
movielocations = pd.read_csv('rawdata/IMDB-Location.list-Parser/movies.csv', header=None,skiprows = 1,sep='|', names=['title','year','a','b','c','location'])
display(movielocations)
movielocations = movielocations.drop(columns=['a','b','c'])
movielocations['year'] = movielocations['year'].fillna(0)
movielocations['year'] = movielocations['year'].astype(int)
movies['year'] = movies['year'].apply(pd.to_numeric, errors='coerce').dropna().astype(int)
movielocations = pd.merge(movies, movielocations)
movielocations = movielocations.drop(columns=['runtime','title','year'])
movielocations = movielocations.rename(columns={"location": "locationListing", "address": "rawAddress"})
movielocations['address'] = movielocations['locationListing']
movielocations['city'] = ''
movielocations['state'] = ''
movielocations['country'] = ''

,title,year,a,b,c,location
0,#1,2010.0,NaN,NaN,NaN,"West Palm Beach, Florida, USA"
1,#1 Cheerleader Camp,2010.0,NaN,V,NaN,"Downtown, Los Angeles, California, USA"
2,#1 Cheerleader Camp,2010.0,NaN,V,NaN,"Malibu, California, USA"
3,#1 Dad,2016.0,NaN,NaN,NaN,"Boston, Massachusetts, USA\t(Filming City)"
4,#1 at the Apocalypse Box Office,2015.0,NaN,NaN,NaN,"Hill End, NSW, Australia"
...,...,...,...,...,...,...
579175,Última vontade,2005.0,NaN,NaN,NaN,"Burela, Lugo, Galicia, Spain"
579176,Últimas imágenes del naufragio,1989.0,NaN,NaN,NaN,"Buenos Aires, Federal District, Argentina"
579177,Últimas memorias vivas,2005.0,NaN,NaN,NaN,"Michoacán, Mexico"
579178,Últimas tardes con Teresa,1984.0,NaN,NaN,NaN,"Barcelona, Catalonia, Spain"


In [7]:
# test geocoding
geolocator = Nominatim(user_agent="moviedatabase")
location = geolocator.geocode("Toronto, Ontario, Canada")
location = geolocator.reverse(str(location.latitude)+","+str(location.longitude))
address = location.raw['address']
print(address)
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')
print('Address :', location)
print('City : ', city)
print('State : ', state)
print('Country : ', country)

{'amenity': 'Toronto City Hall', 'house_number': '100', 'road': 'Queen Street West', 'neighbourhood': 'Financial District', 'quarter': 'Spadina—Fort York', 'city': 'Old Toronto', 'state_district': 'Golden Horseshoe', 'state': 'Ontario', 'postcode': 'M5H 2N2', 'country': 'Canada', 'country_code': 'ca'}
Address : Toronto City Hall, 100, Queen Street West, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
City :  Old Toronto
State :  Ontario
Country :  Canada


In [8]:
geolocator = Nominatim(user_agent="moviedatabase")
for index, row  in movielocations.iterrows():
    location = geolocator.geocode(row['locationListing'], language='en',timeout=None)
    row['rawAddress'] = location
    if hasattr(location,'latitude') and location.latitude is not None and hasattr(location,'longitude') and location.longitude is not None:
        location = geolocator.reverse((location.latitude, location.longitude), language='en',timeout=None)
        if hasattr(location,'raw') and location.raw is not None:
            address = location.raw['address']
            city = address.get('city', '')
            state = address.get('state', '')
            country = address.get('country', '')
            row['city'] = city
            row['state'] = state
            row['country'] = country
movielocations.to_csv('finaldata/shootinglocation.txt', sep='\t', index=False)
display(movielocations)

,movieID,locationListing,rawAddress,city,state,country
0,tt0000574,"Eltham, Victoria, Australia","Eltham, Youth Road, Eltham, Melbourne, Shire o...",Melbourne,Victoria,Australia
1,tt0000574,"Greensborough, Victoria, Australia","Greensborough, Poulter Avenue, Greensborough, ...",Melbourne,Victoria,Australia
2,tt0000574,"Heidelberg, Victoria, Australia","Heidelberg, Melbourne, City of Banyule, Victor...",Melbourne,Victoria,Australia
3,tt0000574,"Melbourne, Victoria, Australia","Melbourne, City of Melbourne, Victoria, Australia",Melbourne,Victoria,Australia
4,tt0000574,"Mitcham, Victoria, Australia","Mitcham, Box Hill to Ringwood Trail, Mitcham, ...",Melbourne,Victoria,Australia
...,...,...,...,...,...,...
262733,tt9884086,"Algarrobo, Chile","Algarrobo, Provincia de San Antonio, Valparais...",NaN,Valparaiso Region,Chile
262734,tt9884086,"Santiago, Chile","Santiago, Provincia de Santiago, Santiago Metr...",Santiago,Santiago Metropolitan Region,Chile
262735,tt9884086,"Port Deposit, Maryland, USA","Port Deposit, Cecil County, Maryland, United S...",NaN,Maryland,United States
262736,tt9887274,"Sacramento, California, USA","(Sacramento, Sacramento County, California, Un...",Sacramento,California,United States


In [9]:
person = pd.read_csv('rawdata/name.basics.tsv/persondata.tsv', sep='\t')
display(person)
person = person.drop(columns=['deathYear','primaryProfession','knownForTitles'])
person = person.rename(columns={'nconst': 'personID', 'primaryName': 'name', 'birthYear': 'birthYear'})
person['birthYear'] = person.birthYear.str.replace(r"\\N", '')
display(person)
person.to_csv('finaldata/person.txt', sep='\t', index=False)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0078723,tt0077975,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0069467,tt0050986,tt0050976"
...,...,...,...,...,...,...
10825471,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10825472,nm9993716,Essias Loberg,\N,\N,NaN,\N
10825473,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10825474,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


,personID,name,birthYear
0,nm0000001,Fred Astaire,1899
1,nm0000002,Lauren Bacall,1924
2,nm0000003,Brigitte Bardot,1934
3,nm0000004,John Belushi,1949
4,nm0000005,Ingmar Bergman,1918
...,...,...,...
10825471,nm9993714,Romeo del Rosario,
10825472,nm9993716,Essias Loberg,
10825473,nm9993717,Harikrishnan Rajan,
10825474,nm9993718,Aayush Nair,


In [10]:
starring = pd.read_csv('rawdata/title.principals.tsv/data.tsv', sep='\t')
display(starring)
starring = starring[(starring.category == 'actor') | (starring.category == 'actress')]
starring = starring.rename(columns={'tconst': "movieID", 'nconst': 'personID'})
starring = pd.merge(starring, person)
starring = starring.drop(columns=['ordering', 'category', 'job', 'birthYear'])
starring['characters'] = starring.characters.str.replace('\\["', '').str.replace('\\"]', '').str.replace('","', ',')
starring = starring.set_index(['movieID', 'personID', 'name']).apply(lambda x: x.str.split(',').explode()).reset_index()   
starring = starring.rename(columns={'characters': 'role'})
starring['role'] = starring.role.str.replace(r"\\N", '')
starring.to_csv('finaldata/starring.txt', sep='\t', index=False)
display(starring)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
43906464,tt9916880,5,nm0996406,director,principal director,\N
43906465,tt9916880,6,nm1482639,writer,\N,\N
43906466,tt9916880,7,nm2586970,writer,books,\N
43906467,tt9916880,8,nm1594058,producer,producer,\N


,movieID,personID,name,role
0,tt0000005,nm0443482,Charles Kayser,Blacksmith
1,tt0000005,nm0653042,John Ott,Assistant
2,tt0000007,nm0179163,James J. Corbett,
3,tt0003116,nm0179163,James J. Corbett,Gentleman Jim
4,tt0003730,nm0179163,James J. Corbett,Danvers
...,...,...,...,...
18255490,tt9916756,nm10781824,Lakeisha Barnes,Lakeisha
18255491,tt9916764,nm10538641,Yvonne Akins,Waitress
18255492,tt9916856,nm10538650,Anna Fitoussi,Sandra
18255493,tt9916856,nm10538646,Andreas Demmel,Stephan


In [11]:
directedby = pd.read_csv('rawdata/title.crew.tsv/directors.tsv', sep='\t')
display(directedby)
directedby = directedby.drop(columns=['writers'])
directedby = directedby.rename(columns={'tconst': "movieID", 'directors': 'personID'})
directedby = pd.merge(directedby, movies)
directedby = directedby.drop(columns=['title','year','runtime'])
directedby = directedby.set_index('movieID').personID.str.split(',', expand=True).stack().reset_index('movieID')
directedby = directedby.rename(columns={0: "personID"})
directedby = pd.merge(directedby, person)
directedby = directedby.drop(columns=['birthYear'])
display(directedby)
directedby.to_csv('finaldata/directedby.txt', sep='\t', index=False)

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7767537,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7767538,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7767539,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7767540,tt9916856,nm10538645,nm6951431


,movieID,personID,name
0,tt0000502,nm0063413,Ricardo de Baños
1,tt0000850,nm0063413,Ricardo de Baños
2,tt0000941,nm0063413,Ricardo de Baños
3,tt0001051,nm0063413,Ricardo de Baños
4,tt0001184,nm0063413,Ricardo de Baños
...,...,...,...
565888,tt9916160,nm5684093,Joost van der Wiel
565889,tt9916170,nm5412267,Tamar Guimaraes
565890,tt9916186,nm10538274,Frank König
565891,tt9916622,nm9272491,Ana Célia de Oliveira


In [12]:
ranking = pd.read_csv('rawdata/tspdt1000.csv', sep=',', encoding='latin-1')
display(ranking)
ranking = ranking[['Const', 'Position']]
ranking = ranking.rename(columns={"Const": "movieID", "Position": "ranking"})
display(ranking)
ranking.to_csv('finaldata/finaldata/ranking.txt', sep='\t', index=False)

,Position,Const,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,1,tt0033467,2021-01-27,2021-01-27,NaN,Citizen Kane,https://www.imdb.com/title/tt0033467/,movie,8.3,119,1941,"Drama, Mystery",408409,1941-05-01,Orson Welles
1,2,tt0052357,2021-01-27,2021-01-27,NaN,Vertigo,https://www.imdb.com/title/tt0052357/,movie,8.3,128,1958,"Mystery, Romance, Thriller",368311,1958-05-09,Alfred Hitchcock
2,3,tt0062622,2021-01-27,2021-01-27,NaN,2001: A Space Odyssey,https://www.imdb.com/title/tt0062622/,movie,8.3,149,1968,"Adventure, Sci-Fi",609733,1968-04-02,Stanley Kubrick
3,4,tt0031885,2021-01-27,2021-01-27,NaN,La règle du jeu,https://www.imdb.com/title/tt0031885/,movie,8.0,110,1939,"Comedy, Drama",26961,1939-07-08,Jean Renoir
4,5,tt0046438,2021-01-27,2021-01-27,NaN,Tôkyô monogatari,https://www.imdb.com/title/tt0046438/,movie,8.2,136,1953,Drama,54186,1953-11-03,Yasujirô Ozu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,tt0013257,2021-01-27,2021-01-27,NaN,Häxan,https://www.imdb.com/title/tt0013257/,movie,7.7,91,1922,"Documentary, Fantasy, History, Horror",12641,1922-09-18,Benjamin Christensen
996,997,tt0029314,2021-01-27,2021-01-27,NaN,Ninjô kami fûsen,https://www.imdb.com/title/tt0029314/,movie,7.7,86,1937,Drama,1852,1937-08-25,Sadao Yamanaka
997,998,tt0096337,2021-01-27,2021-01-27,NaN,Une histoire de vent,https://www.imdb.com/title/tt0096337/,movie,7.0,80,1988,Documentary,767,1988-09-09,Joris Ivens
998,999,tt0166896,2021-01-27,2021-01-27,NaN,The Straight Story,https://www.imdb.com/title/tt0166896/,movie,8.0,112,1999,"Biography, Drama",82871,1999-05-21,David Lynch


,movieID,ranking
0,tt0033467,1
1,tt0052357,2
2,tt0062622,3
3,tt0031885,4
4,tt0046438,5
...,...,...
995,tt0013257,996
996,tt0029314,997
997,tt0096337,998
998,tt0166896,999
